# Quantum Neural Network Image Classifier

The present project aims to apply quantum kernels implemented with IBM’s quantum 
computing framework, Qiskit, to classify simple image patterns. Specifically, distinguishing 
between images of circles and crosses, each represented as 8x8 pixel images.

The central objective of this project is to leverage quantum kernel estimation, an innovative 
method that encodes classical data into quantum states to exploit quantum computational 
advantages, potentially enhancing classification accuracy or reducing computational 
complexity compared to classical approaches.